In [54]:
import pandas as pd
import numpy as np
import CFS

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import SMOTE

In [45]:
df = pd.read_csv('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/commit_guru/ant.csv')
commit_df = pd.read_pickle('/Users/suvodeepmajumder/Documents/AI4SE/git_miner/data/ant_commit.pkl')

In [46]:
df1 = commit_df[commit_df['buggy'] == 1]
X1 = df1.commit_number
X2 = df1.parent
X = np.append(X1,X2)
df = df[df['commit_hash'].isin(X)]
df = df.drop(labels = ['commit_hash', 'author_name', 'author_date_unix_timestamp',
       'author_email', 'author_date', 'commit_message','classification', 'linked', 'contains_bug', 'fixes',
                      'fileschanged','glm_probability', 'rf_probability',
       'repository_id', 'issue_id', 'issue_date', 'issue_type'],axis=1)
df = df.dropna()
df = df[['ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age',
       'nuc', 'exp', 'rexp', 'sexp','fix']]
df = df.astype(np.float64)
df.reset_index(drop=True,inplace=True)

In [47]:
y = df.fix.values
X = df.drop(labels = ['fix'],axis = 1)
X = X.values

In [48]:
z = CFS.cfs(X,y)

In [49]:
cols = df.columns[[z]].tolist()

cols.append('fix')

In [50]:
df = df[cols]

In [56]:
y = df.fix
y=y.astype('bool')
X = df.drop(labels=['fix'],axis=1)
train_X,test_X,train_y,test_y = train_test_split(X, y, test_size=0.33, random_state=13)
print(train_X.shape)
df_new = train_X
df_new['fix'] = train_y
smt = SMOTE.smote(train_X)
df = smt.run()
df.columns = cols
train_y = df.fix
train_y=train_y.astype('bool')
train_X = df.drop(labels=['fix'],axis=1)
print(train_X.shape)
clf = LogisticRegression(penalty='l1')
clf.fit(train_X,train_y)
predicted = clf.predict(test_X)
print(classification_report(test_y, predicted))

(3201, 6)
(3216, 6)
              precision    recall  f1-score   support

       False       0.51      0.96      0.67       781
        True       0.71      0.10      0.18       796

    accuracy                           0.53      1577
   macro avg       0.61      0.53      0.42      1577
weighted avg       0.61      0.53      0.42      1577

